<a href="https://colab.research.google.com/github/vansh7nvc/AI-ML/blob/main/Netflix_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## NETFLIX RECOMMENDATION SYSTEM




In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
# Load the dataset
df = pd.read_csv('/content/netflix_titles.csv')

# Display the first 5 rows
display(df.head())

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [ ]:
df.groupby('type').count()

,show_id,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
type,,,,,,,,,,,
Movie,6131,6131,5943,5656,5691,6131,6131,6129,6128,6131,6131
TV Show,2676,2676,230,2326,2285,2666,2676,2674,2676,2676,2676


In [ ]:
df.dropna(subset=['cast','country','rating'],inplace=True)

In [ ]:
movies = df[df['type'] == 'Movie'].reset_index()
movies = movies.drop(['index', 'show_id', 'type', 'date_added', 'release_year', 'duration', 'description'], axis=1)
movies.head()

,title,director,cast,country,rating,listed_in
0,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...",TV-MA,"Dramas, Independent Movies, International Movies"
1,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,PG-13,"Comedies, Dramas"
2,Je Suis Karl,Christian Schwochow,"Luna Wedler, Jannis Niewöhner, Milan Peschel, ...","Germany, Czech Republic",TV-MA,"Dramas, International Movies"
3,Jeans,S. Shankar,"Prashanth, Aishwarya Rai Bachchan, Sri Lakshmi...",India,TV-14,"Comedies, International Movies, Romantic Movies"
4,Grown Ups,Dennis Dugan,"Adam Sandler, Kevin James, Chris Rock, David S...",United States,PG-13,Comedies


In [ ]:
tv_shows = df.loc[df['type'] == 'TV Show'].reset_index()
tv_shows = tv_shows.drop(['index', 'show_id', 'type', 'date_added', 'release_year', 'duration', 'description'], axis=1)
tv_shows.head()

,title,director,cast,country,rating,listed_in
0,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,TV-MA,"International TV Shows, TV Dramas, TV Mysteries"
1,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,TV-MA,"International TV Shows, Romantic TV Shows, TV ..."
2,The Great British Baking Show,Andy Devonshire,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...",United Kingdom,TV-14,"British TV Shows, Reality TV"
3,Dear White People,NaN,"Logan Browning, Brandon P. Bell, DeRon Horton,...",United States,TV-MA,"TV Comedies, TV Dramas"
4,Falsa identidad,NaN,"Luis Ernesto Franco, Camila Sodi, Sergio Goyri...",Mexico,TV-MA,"Crime TV Shows, Spanish-Language TV Shows, TV ..."


In [ ]:
# Create binary features for 'cast'
actors = movies['cast'].str.split(', ', expand=True).stack()
actors_list = sorted(actors.unique())
binary_actors = movies['cast'].str.get_dummies(sep=', ')

# Create binary features for 'director', handling potential NaN values
movies['director'] = movies['director'].fillna('') # Replace NaN with empty string
directors = movies['director'].str.split(', ', expand=True).stack()
directors_list = sorted(directors.unique())
binary_directors = movies['director'].str.get_dummies(sep=', ')

# Create binary features for 'country', handling potential NaN values
movies['country'] = movies['country'].fillna('') # Replace NaN with empty string
countries = movies['country'].str.split(', ', expand=True).stack()
countries_list = sorted(countries.unique())
binary_countries = movies['country'].str.get_dummies(sep=', ')

# Create binary features for 'listed_in' (genres)
genres = movies['listed_in'].str.split(', ', expand=True).stack()
genres_list = sorted(genres.unique())
binary_genres = movies['listed_in'].str.get_dummies(sep=', ')

# Create binary features for 'rating'
ratings_list = sorted(movies['rating'].unique())
binary_ratings = movies['rating'].str.get_dummies()

display(binary_actors.head())
display(binary_directors.head())
display(binary_countries.head())
display(binary_genres.head())
display(binary_ratings.head())

,Jr.,"""Riley"" Lakdhar Dridi",'Najite Dede,2Mex,4Minute,50 Cent,A. Murat Özgen,A.C. Peterson,A.J. Cook,A.J. Johnson,...,Şebnem Bozoklu,Şebnem Sönmez,Şenay Bozoklar,Şennur Nogaylar,Şerif Sezer,Şevket Çoruh,Şinasi Yurtsever,Şükran Ovalı,Şükrü Özyıldız,Ṣọpẹ́ Dìrísù
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


,A. L. Vijay,A. Raajdheep,A. Salaam,A.R. Murugadoss,Aadish Keluskar,Aamir Bashir,Aamir Khan,Aanand Rai,Aaron Burns,Aaron Hann,...,Ángel Gómez Hernández,Ángeles Reiné,Çagan Irmak,Éric Judor,Éric Toledano,Éric Warin,Ísold Uggadóttir,Óskar Thór Axelsson,Ömer Faruk Sorak,Şenol Sönmez
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


,Afghanistan,Albania,Algeria,Angola,Argentina,Australia,Austria,Bahamas,Bangladesh,Belgium,...,United Arab Emirates,United Kingdom,"United Kingdom,",United States,Uruguay,Vatican City,Venezuela,Vietnam,West Germany,Zimbabwe
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


,Action & Adventure,Anime Features,Children & Family Movies,Classic Movies,Comedies,Cult Movies,Documentaries,Dramas,Faith & Spirituality,Horror Movies,Independent Movies,International Movies,LGBTQ Movies,Movies,Music & Musicals,Romantic Movies,Sci-Fi & Fantasy,Sports Movies,Stand-Up Comedy,Thrillers
0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


,66 min,74 min,84 min,G,NC-17,NR,PG,PG-13,R,TV-14,TV-G,TV-MA,TV-PG,TV-Y,TV-Y7,TV-Y7-FV,UR
0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [ ]:
# Create binary features for 'cast'
tv_shows['cast'] = tv_shows['cast'].fillna('') # Replace NaN with empty string
actors2 = tv_shows['cast'].str.split(', ', expand=True).stack()
actors_list2 = sorted(actors2.unique())
binary_actors2 = tv_shows['cast'].str.get_dummies(sep=', ')

# Create binary features for 'country', handling potential NaN values
tv_shows['country'] = tv_shows['country'].fillna('') # Replace NaN with empty string
countries2 = tv_shows['country'].str.split(', ', expand=True).stack()
countries_list2 = sorted(countries2.unique())
binary_countries2 = tv_shows['country'].str.get_dummies(sep=', ')

# Create binary features for 'listed_in' (genres)
genres2 = tv_shows['listed_in'].str.split(', ', expand=True).stack()
genres_list2 = sorted(genres2.unique())
binary_genres2 = tv_shows['listed_in'].str.get_dummies(sep=', ')

# Create binary features for 'rating'
ratings_list2 = sorted(tv_shows['rating'].unique())
binary_ratings2 = tv_shows['rating'].str.get_dummies()

display(binary_actors2.head())
display(binary_countries2.head())
display(binary_genres2.head())
display(binary_ratings2.head())

,Jr.,'Najite Dede,2 Chainz,A Boogie Wit tha Hoodie,A.D. Miles,A.J. Cook,A.J. LoCascio,AFRA,AJ Michalka,AJ Rivera,...,İnanç Konukçu,İpek Bilgin,İpek Erdem,İpek Filiz Yazıcı,İsmail Filiz,İştar Gökseven,Şafak Başkaya,Şehsuvar Aktaş,Şenay Gürler,Şükrü Özyıldız
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


,Argentina,Australia,Austria,Azerbaijan,Belarus,Belgium,Brazil,Canada,Chile,China,...,Switzerland,Syria,Taiwan,Thailand,Turkey,Ukraine,United Arab Emirates,United Kingdom,United States,West Germany
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


,Anime Series,British TV Shows,Classic & Cult TV,Crime TV Shows,Docuseries,International TV Shows,Kids' TV,Korean TV Shows,Reality TV,Romantic TV Shows,...,Stand-Up Comedy & Talk Shows,TV Action & Adventure,TV Comedies,TV Dramas,TV Horror,TV Mysteries,TV Sci-Fi & Fantasy,TV Shows,TV Thrillers,Teen TV Shows
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
1,0,0,0,0,0,1,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


,NR,R,TV-14,TV-G,TV-MA,TV-PG,TV-Y,TV-Y7,TV-Y7-FV
0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,1,0,0,0,0
2,0,0,1,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0
4,0,0,0,0,1,0,0,0,0


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Combine binary features for movies and TV shows, aligning columns
combined_binary = pd.concat([binary_actors, binary_directors, binary_countries, binary_genres, binary_ratings], axis=1)
combined_binary2 = pd.concat([binary_actors2, binary_countries2, binary_genres2, binary_ratings2], axis=1) # No director for TV shows

# Align columns of combined_binary2 with combined_binary
combined_binary2 = combined_binary2.reindex(columns=combined_binary.columns, fill_value=0)

combined_features = pd.concat([combined_binary, combined_binary2], ignore_index=True)


# Calculate the cosine similarity matrix
cosine_sim_matrix = cosine_similarity(combined_features)

# Create a combined dataframe with titles for easy lookup
combined_titles = pd.concat([movies['title'], tv_shows['title']], ignore_index=True)


def recommender(search):
    if search not in combined_titles.values:
        return "Title not in dataset. Please check spelling."

    # Get the index of the search title in the combined dataframe
    idx = combined_titles[combined_titles == search].index[0]

    # Get the cosine similarity scores for the search title
    sim_scores = list(enumerate(cosine_sim_matrix[idx]))

    # Sort the titles based on similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the top 5 most similar titles (excluding the search title itself)
    sim_scores = sim_scores[1:6]

    # Get the indices of the top titles
    top_indices = [i[0] for i in sim_scores]

    # Return the top 5 recommended titles
    return combined_titles.iloc[top_indices]

In [ ]:
recommender('The Conjuring')

,title
603,The Conjuring 2
3223,Creep
507,Insidious
3191,Hush
3284,13 Sins


In [ ]:
recommender('Hook')

,title
5375,Girl from Nowhere
5424,The Judgement
5504,The Underclass
6105,Bangkok Buddies
6497,Project S The Series


In [ ]:
recommender('Still')

,title
4450,Oh My Ghost 2
5093,The Unborn Child
5828,Hook
4451,Oh My Ghost 3
2204,Nang Nak


In [ ]:
recommender('Nang Nak')

,title
6244,Sleepless Society: Nyctophobia
5375,Girl from Nowhere
5424,The Judgement
5504,The Underclass
6105,Bangkok Buddies


In [ ]:
recommender('Kota Factory')

,title
5952,Ladies Up
6123,Little Things
6184,Engineering Girls
6185,Girls Hostel
6186,Inmates
